In [117]:
import rooms.dbmanager as dbm
from datetime import datetime

import pandas as pd
from pony.orm import select, show

In [118]:
db = dbm.connect("rooms.sqlite")

In [119]:
select(rd.draw_year for rd in db.RoomDraw)[:]

[2016, 2013]

In [120]:
mathey_start_draw  = datetime.strptime("04/18/18 09:00 AM", "%m/%d/%y %H:%M %p")
my_mathey_drawtime = datetime.strptime("04/19/18 10:16 AM", "%m/%d/%y %H:%M %p")

upperclass_start_draw = datetime.strptime("04/30/18 09:00 AM", "%m/%d/%y %I:%M %p")
my_upperclass_drawtime = datetime.strptime("05/03/18 04:36 PM", "%m/%d/%y %I:%M %p")
my_upperclass_delta = my_upperclass_drawtime - upperclass_start_draw

In [121]:
def unix2timedelta(unix):
    days = unix // 86400
    unix = unix % 86400
    hours = unix // 3600
    unix = unix % 3600
    minutes = unix // 60
    unix = unix % 60
    seconds = unix
    return (days, hours, minutes, seconds)

def room2time(room):
    draws = room.drawings.select()[:]
    for rd in draws:
        if rd.draw_year == 2016:
            return rd.timefromstart
    return None

def compute_min(college="Upperclass"):
    times = select(
        rd.timefromstart 
        for rd in db.RoomDraw 
        if rd.draw_year == 2016 and rd.room.college == college
    )[:]
    return min(times)

def later_rooms(rooms, min_time_in_draw, my_timedelta_in_draw):
    times = [room2time(room) for room in rooms]
    rooms = [r for r, t in zip(rooms, times) if t is not None]
    times = [t for t in times if t is not None]
    adjusted = [t - min_time_in_draw for t in times]
    tds = [unix2timedelta(t) for t in adjusted]
    is_later = [td >= my_timedelta_in_draw for td in tds]
    return [r for r, l in zip(rooms, is_later) if l]

def tabular(rooms, min_time_in_draw):
    dicts = []
    for r in rooms: 
        d = r.to_dict()
        if room2time(r) is not None:
            d["delta"]  = unix2timedelta(room2time(r) - min_time_in_draw)
        dicts.append(d)
    return pd.DataFrame(dicts).sort_values("delta")

# Upperclass

In [126]:
query_rooms = select(r for r in db.Room if r.college == "Upperclass" and r.occupancy == 3)[:]
min_time = compute_min(college="Upperclass")
my_delta = unix2timedelta(my_upperclass_delta.total_seconds())
print("You draw %dd %dh %dm %ds after the start." % my_delta)
possibles = later_rooms(query_rooms, min_time, my_delta)

print("%d / %d" % (len(possibles), len(query_rooms)))

You draw 3d 7h 36m 0s after the start.
45 / 115


In [128]:
tabular(possibles, min_time)

,building,college,delta,floor,id,numrooms,occupancy,reserved,roomnum,sqft,subfree
38,Patton Hall,Upperclass,"(3, 8, 15, 53)",2,1306,3,3,False,063,430,False
33,Laughlin Hall,Upperclass,"(3, 8, 31, 36)",1,1133,3,3,False,616,409,False
7,Cuyler Hall,Upperclass,"(3, 8, 32, 4)",1,576,3,3,False,211,425,False
4,1903 Hall,Upperclass,"(4, 0, 5, 23)",2,112,3,3,False,321,393,False
5,1903 Hall,Upperclass,"(4, 0, 5, 25)",2,115,3,3,False,324,389,False
27,Henry Hall,Upperclass,"(4, 0, 19, 5)",3,1032,3,3,False,232,454,False
39,Patton Hall,Upperclass,"(4, 0, 25, 39)",1,1330,3,3,False,T03,419,False
11,Cuyler Hall,Upperclass,"(4, 0, 55, 15)",1,587,3,3,False,312,415,False
16,Foulke Hall,Upperclass,"(4, 1, 17, 17)",1,896,3,3,False,114,441,False
44,Wright Hall,Upperclass,"(4, 1, 39, 49)",4,1846,3,3,False,407,429,False


In [78]:
show(possibles[0])

instance of Room
id|reserved|college   |building |floor|roomnum|sqft|occupancy|numrooms|subfree
--+--------+----------+---------+-----+-------+----+---------+--------+-------
9 |False   |Upperclass|1901 Hall|1    |114    |191 |1        |1       |False  


# Mathey

In [ ]:
mathey_singles = select(r for r in db.Room)